# Main run script

This script contains the main procedure to calculate global root zone storage capacities. 

This scripts only works in the conda environment **sr_env**. In this environment all required packages are available. If you have **not** installed and activated this environment before opening this script, you should check the installation section in the *README* file. 
 

### 1. Getting started
First, import all the required packages.

In [ ]:
# import packages
import glob
from pathlib import Path
import os
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import calendar
import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt

from scipy.optimize import least_squares
import sklearn
from sklearn.linear_model import LinearRegression
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

Here we import all the python functions defined in the scripts *f_catch_characteristics.py* and *f_preprocess_discharge.py*.

In [ ]:
# import python functions
from f_catch_characteristics import *
from f_preprocess_discharge import *
from f_sr_calculation import *
from f_regression import *

### 2. Define working directory
Here we define the working directory, where all the scripts and data are saved. Make sure that you generate within this working directory the following subdirectories with the data:\
/work_dir/data/forcing/*netcdf forcing files*\
/work_dir/data/shapes/*catchment shapefiles*\
/work_dir/data/gsim_discharge/*gsim discharge timeseries*


In [ ]:
# define your working directory
# work_dir=Path("/home/fransjevanoors/global_sr_module")
work_dir=Path("/work/users/vanoorschot/fransje/scripts/GLOBAL_SR/global_sr_module")

Here we create the output directory inside your working directory. In the remainder of this module, the same command will be used regularly to create directories.

In [ ]:
# make output directory
if not os.path.exists(f'{work_dir}/output'):
    os.makedirs(f'{work_dir}/output')

### 3. Make lists of catchment IDs
The module calculates catchment root zone storage capacities for a large sample of catchments. Here we save the catchment names in a .txt file, for later use in the scripts.

In [ ]:
# list the filenames of catchment shapefiles
shape_dir = Path(f'{work_dir}/data/shapes/')
shapefiles = glob.glob(f"{shape_dir}/*shp")

# make an empty list
catch_id_list = []

# loop over the catchment shapefiles, extract the catchment id and store in the empty list
for i in shapefiles:
    catch_id_list.append(Path(i).name.split('.')[0])
    
# save the catchment id list in your output directory
np.savetxt(f'{work_dir}/output/catch_id_list.txt',catch_id_list,fmt='%s')

In [ ]:
print(catch_id_list)

### 4. Preprocess GSIM discharge data

The GSIM yearly discharge timeseries are stored in *.year* files. A detailed explanation of the column names is provided in Table 3 and 4 in https://essd.copernicus.org/articles/10/787/2018/. Here we preprocess these data into readable *.csv* files for each catchment. The preprocessing function *preprocess_gsim_discharge* is defined in the file *f_preprocess_discharge.py*. With this function we generate for each catchment a file with the yearly discharge timeseries and a file with the specifications of the catchment.

In [ ]:
# make output directories
if not os.path.exists(f'{work_dir}/output/discharge/timeseries'):
    os.makedirs(f'{work_dir}/output/discharge/timeseries')
    
if not os.path.exists(f'{work_dir}/output/discharge/characteristics'):
    os.makedirs(f'{work_dir}/output/discharge/characteristics')

In [ ]:
# define folder with discharge timeseries data
fol_in = f'{work_dir}/data/gsim_discharge/'

# define output folder
fol_out = f'{work_dir}/output/discharge/'

# run preprocess_gsim_discharge function (defined in f_preprocess_discharge.py) for all catchments in catch_id_list
for catch_id in catch_id_list:
    preprocess_gsim_discharge(catch_id, fol_in, fol_out)

### 5. From gridded data to catchment timeseries
For this step go to the notebook *run_script_grid_to_catchments*. This part is run in another notebook. The output data of this script can be found in *work_dir/output/forcing_timeseries*.

### 6. Google earth engine for catchment characteristics
For this step go to the notebook *run_script_earthengine*. This part is run in another notebook. The output data of this script can be found in *work_dir/output/earth_engine_timeseries*.

### 7. Catchment descriptor variables
For the global root zone storage capacity estimation, we need to calculate catchment descriptor variables. These descriptors can be climatological variables (e.g. mean precipitation (p_mean); seasonality of precipitation (si_p); timelag between maximum P and Ep (phi)) or landscape variables (e.g. mean treecover (tc); mean elevation (h_mean)). A detailed list of all the descriptors considered is provided here xxxxx.\
To calculate the catchment descriptor variables we use the *catch_characteristics* function from the *f_catch_characteristics.py* file. In this function you specify the variables of interest, the catchment ID and your in- and output folders. Then, based on all the timeseries you have generated in the preceding codes it will return a table with the catchment descriptor variables for all your catchments (that is saved as csv in your *work_dir/catchment_characteristics.csv*).

In [ ]:
# define in and output folder
fol_in=f'{work_dir}/output/'
fol_out=f'{work_dir}/output/'

# define variables of interest
var=['p_mean','ep_mean','q_mean','t_mean','ai','si_p','si_ep','phi','tc','ntc','nonveg']

# run catch_characteristics (defined in f_catch_characteristics.py) for the catchments in your catch_id_list
catch_characteristics(var, catch_id_list, fol_in, fol_out)

In [ ]:
# combine catchment geometries in single shapefile
shape_dir = f'{work_dir}/data/shapes/'
out_dir = f'{work_dir}/output'
geo_catchments(shape_dir,out_dir)

### 8. Calculate root zone storage capacity
Here we calculate the catchment root zone storage capacity (Sr) based on catchment water balances. First, catchment root zone storage deficits (Sd) are computed as the cumulative difference between P and Et (transpiration). The result of one catchment is visualised in a figure. Second, the Sr is then calculated based on an extreme value analysis of the storage deficits for different return periods. A detailed description of this method can be found here xxxxxx.

Here we use the *run_sd_calculation* and *run_sr_calculation* functions from the *f_sr_calculation* file. The Sd result of one catchment is visualised using *plot_sd*. The Sr results are merged using the *merge_sr* function and visualised using the *plot_sr* function. The output of both storage deficit and Sr calculations are saved in your *work_dir/output/sr_calculation*.


In [ ]:
# make output directories
if not os.path.exists(f'{work_dir}/output/sr_calculation/sd_catchments'):
    os.makedirs(f'{work_dir}/output/sr_calculation/sd_catchments')
    
if not os.path.exists(f'{work_dir}/output/sr_calculation/sr_catchments'):
    os.makedirs(f'{work_dir}/output/sr_calculation/sr_catchments')

Calculate storage deficits using the *run_sd_calculation* function from *f_sr_calculation*

In [ ]:
# define directories
pep_dir = f'{work_dir}/output/forcing_timeseries/processed/daily'
q_dir = f'{work_dir}/output/discharge/timeseries'
out_dir = f'{work_dir}/output/sr_calculation/sd_catchments'

# run sd calculation for all catchments in catch_id_list
for catch_id in catch_id_list:
    run_sd_calculation(catch_id, pep_dir, q_dir, out_dir)
    
# plot sd example - use the first catchment from catch_id_list
sd_dir = f'{work_dir}/output/sr_calculation/sd_catchments'
plot_sd(catch_id_list[0], sd_dir)

Calculate Sr using the *run_sr_calculation* function from *f_sr_calculation* and merge the catchment Sr values into one dataframe with *merge_sr_catchments*. The functions return a table with the catchment Sr values for the different return periods.

In [ ]:
# define directories
sd_dir = f'{work_dir}/output/sr_calculation/sd_catchments'
out_dir = f'{work_dir}/output/sr_calculation/sr_catchments'

# define return periods
rp_array = [2,3,5,10,20,30,40,50,60]

# run sr calculation for all catchments in catch_id_list
for catch_id in catch_id_list:
    run_sr_calculation(catch_id, rp_array, sd_dir, out_dir)
    
# merge catchment sr dataframes into one dataframe
sr_dir = f'{work_dir}/output/sr_calculation/sr_catchments'
out_dir = f'{work_dir}/output/sr_calculation/'
merge_sr_catchments(sr_dir,out_dir)

Mapping Sr using the *plot_sr* function from *f_sr_calculation*

In [ ]:
sr_file = f'{work_dir}/output/sr_calculation/sr_all_catchments.csv'
shp_file = f'{work_dir}/output/geo_catchments.shp'
rp=20
plot_sr(shp_file,sr_file,rp)

### 9. Regression
Here we run the linear regression model to predict the catchment Sr values based on the descriptor parameters. We use the *f_regression* function to calculate the linear regression parameters for the considered catchments.
We use the treecover data to separate the regression for high and low vegetation, the threshold values for tree cover (tc), non tree cover (ntc) and no-vegetation (nonveg) define this separation.

The output is a figure showing the estimated (step 8) and predicted (from regression) Sr values and a table with the regression parameter values, some statistics for the regression performance and the threshold values for tree cover. 

In [ ]:
# read the catchment characteristics and sr tables
cc_df = pd.read_csv(f'{work_dir}/output/catchment_characteristics.csv',index_col=0)
sr_df = pd.read_csv(f'{work_dir}/output/sr_calculation/sr_all_catchments.csv',index_col=0)

# define the descriptor variables
dpar = ['p_mean','ep_mean','t_mean','si_p']

# return period of Sr estimate
rp = 20

# define the vegetation thresholds for the regression
tc_th, ntc_th, nonveg_th = 10, 0, 0

# run the regression (r_regression in f_regression.py)
run_regression(cc_df, sr_df, dpar, rp, tc_th, ntc_th, nonveg_th)